# **Food Delivery Data Cleaning**

# 概要

このプロジェクトでは、メルボルンのレストランの架空のフードデリバリーデータを対象に、データの異常を検出し修正します。データには、日付、注文内容、価格、会員ステータス（ロイヤルティ会員 or 非会員）、配達距離 などの情報が含まれています。

### **処理の流れ**
このノートブックでは、以下の順番でデータのクリーニングを行います。

1. 日付の修正
2. 時間の修正
3. 支店コードの修正
4. メニュー情報の修正
5. 位置情報の修正
6. 配達距離の修正
7. 会員ステータスの修正

これらの処理を行い、最終的に データ分析や機械学習モデルの学習に適したデータを作成します。

# 1 環境準備

## ライブラリのインポート

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from datetime import datetime
from datetime import time
import ast
import re
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## データの読み込み

In [ ]:
# データの読み込み
dirty_data = pd.read_csv('data/dirty_data.csv', encoding = 'utf-8')
missing_data = pd.read_csv('data/missing_data.csv',encoding = 'utf-8')
outlier_data = pd.read_csv('data/outlier_data.csv', encoding = 'utf-8')

branch_data = pd.read_csv('data/branches.csv', encoding = 'utf-8')
edge_data = pd.read_csv('data/edges.csv', encoding = 'utf-8')
node_data = pd.read_csv('data/nodes.csv', encoding = 'utf-8')
suburb_data = pd.read_excel('data/suburb_info.xlsx')

In [3]:
# データの確認
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDI10384,2018-01-16,08:30:25,Breakfast,NS,"[('Coffee', 6), ('Cereal', 6)]",171.00,37.806118,144.934214,1,10.009,7.788453
1,ORDB06250,2018-04-16,11:22:49,Breakfast,TP,"[('Eggs', 1), ('Coffee', 6), ('Pancake', 2)]",115.50,-37.810433,144.968155,1,8.762,11.704791
2,ORDC03391,2018-11-12,18:28:43,Dinner,NS,"[('Pasta', 8), ('Shrimp', 5), ('Salmon', 8), (...",1028.00,-37.803893,144.978943,0,7.129,13.074209
3,ORDZ10295,2018-08-25,15:46:28,Lunch,NS,"[('Burger', 4), ('Steak', 6)]",394.00,-37.820483,144.981167,1,9.603,16.994845
4,ORDK02931,2018-12-14,19:29:34,Dinner,BK,"[('Salmon', 10), ('Pasta', 1)]",437.50,-37.817963,144.932101,0,10.610,17.507001
...,...,...,...,...,...,...,...,...,...,...,...,...
495,ORDK02085,2018-09-05,16:06:45,Dinner,BK,"[('Fish&Chips', 8), ('Pasta', 9), ('Salmon', 8)]",855.50,-37.800097,144.953861,0,8.632,15.506404
496,ORDB09078,12-10-2018,17:27:53,Dinner,TP,"[('Salmon', 6), ('Pasta', 1), ('Fish&Chips', 3...",810.50,-37.818915,144.964563,0,7.846,12.361560
497,ORDC10427,2018-01-27,08:50:42,Breakfast,NS,"[('Pancake', 8), ('Eggs', 1), ('Coffee', 7)]",767.20,-37.821267,144.957756,0,9.293,16.673407
498,ORDI10080,2018-03-08,10:11:49,Breakfast,NS,"[('Eggs', 2), ('Pancake', 9), ('Cereal', 3), (...",370.25,-37.818757,144.983192,0,9.035,13.514620


In [4]:
missing_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDJ11041,2018-06-20,13:34:38,Lunch,NaN,"[('Steak', 9), ('Fries', 6), ('Burger', 10), (...",1148.60,-37.800838,144.972446,0,10.036,13.532827
1,ORDI02055,2018-08-26,15:46:28,Lunch,NS,"[('Chicken', 2), ('Burger', 6), ('Steak', 1)]",295.00,-37.813013,144.938946,0,10.272,NaN
2,ORDJ00269,2018-11-15,18:28:43,Dinner,TP,"[('Pasta', 5), ('Fish&Chips', 8), ('Shrimp', 2)]",525.50,-37.807592,144.945359,0,10.089,13.805427
3,ORDX06001,2018-01-13,08:20:16,Breakfast,BK,"[('Coffee', 6), ('Pancake', 10), ('Cereal', 7)]",434.50,-37.817017,144.935686,1,10.195,9.273274
4,ORDJ01922,2018-11-02,18:08:27,Dinner,TP,"[('Fish&Chips', 7), ('Shrimp', 1), ('Pasta', 3)]",381.50,-37.810403,144.999086,0,10.959,14.536421
...,...,...,...,...,...,...,...,...,...,...,...,...
495,ORDK05906,2018-09-10,16:16:54,Dinner,BK,"[('Pasta', 4), ('Shrimp', 6)]",434.00,-37.800352,144.989143,0,6.051,NaN
496,ORDI06900,2018-04-13,11:22:49,Breakfast,NaN,"[('Eggs', 6), ('Coffee', 5), ('Cereal', 9), ('...",455.50,-37.825935,145.007148,0,NaN,16.297903
497,ORDZ10456,2018-09-16,16:37:10,Dinner,NS,"[('Shrimp', 10), ('Salmon', 4), ('Pasta', 9), ...",1126.50,-37.804138,144.983392,0,7.513,15.625400
498,ORDB10191,2018-05-23,12:43:56,Lunch,TP,"[('Fries', 3), ('Burger', 1), ('Steak', 7), ('...",638.00,-37.814045,144.990019,0,10.033,13.395150


In [5]:
outlier_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDX07128,2018-03-27,10:42:15,Breakfast,BK,"[('Eggs', 9), ('Coffee', 3), ('Cereal', 3)]",283.50,-37.812458,144.953838,0,8.327,12.705625
1,ORDY04820,2018-07-08,14:15:12,Lunch,TP,"[('Steak', 8), ('Salad', 1)]",377.20,-37.808446,144.976785,0,9.262,13.680240
2,ORDI06977,2018-03-15,10:21:58,Breakfast,NS,"[('Eggs', 6), ('Pancake', 4), ('Cereal', 8), (...",449.50,-37.807687,144.970591,0,7.015,11.968503
3,ORDC04090,2018-02-10,09:21:07,Breakfast,NS,"[('Pancake', 9), ('Coffee', 7), ('Cereal', 7)]",417.75,-37.807757,144.965955,0,7.099,14.584832
4,ORDC04240,2018-02-04,09:00:50,Breakfast,NS,"[('Coffee', 2), ('Eggs', 1), ('Cereal', 9), ('...",250.25,-37.817336,144.991590,0,9.405,15.912437
...,...,...,...,...,...,...,...,...,...,...,...,...
495,ORDA06832,2018-05-07,12:03:22,Lunch,BK,"[('Fries', 4), ('Steak', 4), ('Chicken', 2), (...",343.60,-37.807006,144.955015,0,8.258,21.757807
496,ORDJ07556,2018-03-14,10:21:58,Breakfast,TP,"[('Eggs', 7), ('Pancake', 5), ('Coffee', 4)]",305.25,-37.813032,144.951469,0,9.080,11.376769
497,ORDX05898,2018-11-07,18:18:35,Dinner,BK,"[('Fish&Chips', 5), ('Shrimp', 7), ('Salmon', ...",963.00,-37.809163,144.960850,0,7.742,14.690451
498,ORDY00078,2018-11-21,18:49:00,Dinner,TP,"[('Shrimp', 3), ('Salmon', 6)]",408.00,-37.825000,144.966907,0,7.984,12.906137


In [6]:
branch_data

,branch_code,branch_name,branch_lat,branch_lon
0,NS,Nickolson,-37.773803,144.983647
1,TP,Thompson,-37.861835,144.905716
2,BK,Bakers,-37.815834,145.046450


In [7]:
edge_data

,Unnamed: 0,u,v,distance(m),street type,speed(km/h)
0,0,711327755,711332946,58,1,15
1,2,711327755,55725841,15,2,20
2,3,711327755,711327760,165,1,15
3,4,711327756,703557073,4,0,10
4,5,711327756,711332946,8,1,15
...,...,...,...,...,...,...
42219,50006,1449431614,1449431244,16,2,20
42220,50007,1449431614,1449431574,17,2,20
42221,50008,1449431622,60095777,196,1,15
42222,50009,1449431623,1449431622,19,1,15


In [8]:
node_data

,node,lat,lon
0,711327755,-37.807675,144.955873
1,711327756,-37.807094,144.955978
2,711327759,-37.807301,144.957817
3,711327760,-37.807885,144.957719
4,777781264,-37.805221,144.952173
...,...,...,...
17112,767688602,-37.804021,144.969444
17113,777781181,-37.805238,144.952321
17114,1449431614,-37.815345,144.974603
17115,1449431622,-37.815262,144.975149


In [9]:
suburb_data

,suburb,number_of_houses,number_of_units,municipality,aus_born_perc,median_income,median_house_price,population
0,ABBOTSFORD,2304,4706,Yarra,68%,"$1,797","$1,299,400",4025
1,ABERFELDIE,1410,453,Moonee Valley,81%,"$1,571","$1,926,600",22442
2,ALBANVALE,1897,138,Brimbank,46%,$907,"$594,200",54005
3,ALBION,1389,1392,Brimbank,52%,$929,"$739,100",30677
4,ALPHINGTON,1729,1099,Darebin,73%,"$1,538","$1,729,600",9227
...,...,...,...,...,...,...,...,...
197,WILLIAMS LANDING,2735,173,Wyndham,87%,"$1,842","$866,400",170
198,WINDSOR,2201,4448,Stonnington,66%,"$1,560","$1,629,600",17776
199,WOLLERT,6516,259,Whittlesea,80%,"$1,355","$704,700",350
200,YALLAMBIE,1286,81,Banyule,79%,"$1,458","$998,200",12063


# 2 データの異常検知と修正

**前提**
- 1つの行（注文）は1つのエラーしか持たない
- missing_data.csv には、データの欠損や不完全な情報に関する異常のみが含まれており、それ以外のデータ異常はない
- 同様に、outlier_data.csv には、外れ値（outliers）以外のデータ異常は含まれていない

## 2.1 日付の修正
日付フォーマットの不整合を修正

In [10]:
dirty_data['date']

0      2018-01-16
1      2018-04-16
2      2018-11-12
3      2018-08-25
4      2018-12-14
          ...    
495    2018-09-05
496    12-10-2018
497    2018-01-27
498    2018-03-08
499    2018-26-07
Name: date, Length: 500, dtype: object

データの「date」列には複数のフォーマット（"%Y-%m-%d", "%Y-%d-%m", "%d-%m-%Y" など）が混在している。

これを統一し、フォーマットの不整合を修正する。

In [11]:
# 修正された行のインデックスを記録
fixed_index = []

def detect_incorrect_date_format(df, error_index_list, column='date'):
    """
    不正な日付フォーマットを検出し、修正対象の行を記録する。

    Args:
        df (pd.DataFrame): 日付データを含むDataFrame。
        error_index_list (list): 不正なフォーマットの行インデックスを保存するリスト。
        column (str): チェック対象の日付カラム名。

    Returns:
        list: 不正なフォーマットの行インデックスリスト。
    """
    for idx, row in df.iterrows():
        try:
            # 正しいフォーマット ("%Y-%m-%d") でパースできるか確認
            datetime.strptime(row[column], '%Y-%m-%d')
        except ValueError:
            # エラーが発生したら、その行のインデックスを記録
            error_index_list.append(idx)
    return error_index_list

# 不正な日付フォーマットのレコードを検出
detect_incorrect_date_format(dirty_data, fixed_index)

[19,
 31,
 40,
 54,
 75,
 79,
 125,
 131,
 156,
 165,
 187,
 191,
 195,
 211,
 214,
 236,
 252,
 260,
 261,
 310,
 346,
 349,
 375,
 376,
 377,
 392,
 409,
 414,
 430,
 431,
 433,
 448,
 455,
 462,
 493,
 496,
 499]

In [12]:
# このコードは My Data Talk (2022) を参考にしています。
# 日付フォーマットを統一 ("%Y-%m-%d" に変換)
dirty_data['date'] = pd.to_datetime(dirty_data['date'], \
                                    infer_datetime_format = True, dayfirst=True)
dirty_data['date'] = dirty_data['date'].dt.strftime('%Y-%m-%d')

# 修正完了メッセージ
print("日付フォーマットを統一しました。")

日付フォーマットを統一しました。


In [13]:
# 修正されたレコードの合計数を表示
print(f"\n合計 {len(set(fixed_index))} 件の配達履歴を修正しました。")


合計 37 件の配達履歴を修正しました。


In [14]:
dirty_data['date']

0      2018-01-16
1      2018-04-16
2      2018-11-12
3      2018-08-25
4      2018-12-14
          ...    
495    2018-09-05
496    2018-10-12
497    2018-01-27
498    2018-03-08
499    2018-07-26
Name: date, Length: 500, dtype: object

## 2.2 時間の修正
注文時間と食事区分の不一致を修正

**前提**
- time 列の値は常に正しく記録されている（修正の必要なし）

- 食事区分のルール
    - 朝食（Breakfast） → 08:00 〜 12:00
    - 昼食（Lunch） → 12:00:01 〜 16:00
    - 夕食（Dinner） → 16:00:01 〜 20:00

※ 各注文の order_type が、この時間帯と一致しているかを確認し、誤っている場合は修正する。

In [15]:
# 'time' 列の基本的な情報
print("【time列の基本統計情報】")
print(dirty_data['time'].describe())

# 'order_type' 列の基本的な情報
print("\n【order_type列の基本統計情報】")
print(dirty_data['order_type'].describe())

# 'order_type' 列のユニークな値のカウント
print("\n【order_type列の値の出現回数】")
print(dirty_data['order_type'].value_counts())

【time列の基本統計情報】
count          500
unique          72
top       12:23:39
freq            16
Name: time, dtype: object

【order_type列の基本統計情報】
count           500
unique            3
top       Breakfast
freq            174
Name: order_type, dtype: object

【order_type列の値の出現回数】
Breakfast    174
Lunch        165
Dinner       161
Name: order_type, dtype: int64


In [16]:
# 'time' 列を時刻データに変換
dirty_data['time'] = pd.to_datetime(dirty_data['time'], format='%H:%M:%S').dt.time
print(type(dirty_data['time'].iloc[0]))  # 確認

<class 'datetime.time'>


In [17]:
# 食事区分のルールを定義
breakfast_start, breakfast_end = time(8, 0, 0), time(12, 0, 0)
lunch_start, lunch_end = time(12, 0, 1), time(16, 0, 0)
dinner_start, dinner_end = time(16, 0, 1), time(20, 0, 0)

# 修正前の各食事区分の件数を確認
print("修正前の注文タイプの分布:")
print(dirty_data['order_type'].value_counts())

# 各時間帯で誤った食事区分があるか確認
print("\n誤分類された注文の確認:")
for order_type, start, end in [('Breakfast', breakfast_start, breakfast_end),
                               ('Lunch', lunch_start, lunch_end),
                               ('Dinner', dinner_start, dinner_end)]:
    mask = (dirty_data['time'] > start) & (dirty_data['time'] < end)
    misclassified = dirty_data[mask & (dirty_data['order_type'] != order_type)]
    print(f"\n{order_type}（{start} 〜 {end}）の時間帯に誤分類された注文:")
    print(misclassified[['time', 'order_type']].head())

修正前の注文タイプの分布:
Breakfast    174
Lunch        165
Dinner       161
Name: order_type, dtype: int64

誤分類された注文の確認:

Breakfast（08:00:00 〜 12:00:00）の時間帯に誤分類された注文:
         time order_type
107  10:32:06     Dinner
159  11:02:32      Lunch
175  09:51:32      Lunch
225  11:12:40     Dinner
284  09:00:50     Dinner

Lunch（12:00:01 〜 16:00:00）の時間帯に誤分類された注文:
        time order_type
6   12:23:39  Breakfast
16  15:05:54     Dinner
22  14:35:29     Dinner
36  12:54:05  Breakfast
51  14:25:21  Breakfast

Dinner（16:00:01 〜 20:00:00）の時間帯に誤分類された注文:
         time order_type
95   19:09:17  Breakfast
105  17:17:44      Lunch
181  18:18:35  Breakfast
215  16:57:27  Breakfast
231  18:18:35      Lunch


In [18]:
# 食事区分を修正する関数
def correct_order_type(df, fixed_indexes):
    """
    各注文の時間帯を確認し、適切な食事区分に修正する。

    Args:
        df (pd.DataFrame): 注文データを含むDataFrame
        fixed_indexes (list): 修正されたレコードのインデックスを保存するリスト
    """
    for idx, row in df.iterrows():
        order_time = row['time']

        # 注文時間が朝食の時間帯で、order_typeが誤っている場合
        if breakfast_start <= order_time <= breakfast_end:
            if row['order_type'] != 'Breakfast':
                df.at[idx, 'order_type'] = 'Breakfast'
                fixed_indexes.append(idx)

        # 注文時間が昼食の時間帯で、order_typeが誤っている場合
        elif lunch_start <= order_time <= lunch_end:
            if row['order_type'] != 'Lunch':
                df.at[idx, 'order_type'] = 'Lunch'
                fixed_indexes.append(idx)

        # 注文時間が夕食の時間帯で、order_typeが誤っている場合
        elif dinner_start <= order_time <= dinner_end:
            if row['order_type'] != 'Dinner':
                df.at[idx, 'order_type'] = 'Dinner'
                fixed_indexes.append(idx)

# 食事区分の修正を適用
correct_order_type(dirty_data, fixed_index)

# 修正後の食事区分の分布を確認
print("修正後の食事区分の分布:")
print(dirty_data['order_type'].value_counts())

# 修正されたレコードの合計数を表示
print(f"\n合計 {len(set(fixed_index))} 件の配達履歴を修正しました。")

修正後の食事区分の分布:
Breakfast    173
Lunch        168
Dinner       159
Name: order_type, dtype: int64

合計 74 件の配達履歴を修正しました。


## 2.3 支店コードの修正

**前提**
- order_id 列の値は常に正しく記録されている（修正の必要なし）

### 支店コードのフォーマットを統一

In [19]:
# 各支店コードの登場回数をカウント
print("各支店コードの出現回数:")
print(dirty_data['branch_code'].value_counts())

各支店コードの出現回数:
NS    162
TP    160
BK    148
ns     14
bk      9
tp      7
Name: branch_code, dtype: int64


In [20]:
# すべての支店コードを大文字に変換してフォーマットを統一
dirty_data['branch_code'] = dirty_data['branch_code'].str.upper()

# 修正後の支店コードの分布を確認
print("修正後の各支店コードの出現回数:")
print(dirty_data['branch_code'].value_counts())

修正後の各支店コードの出現回数:
NS    176
TP    167
BK    157
Name: branch_code, dtype: int64


### 誤った支店コードを修正

**支店ごとの order_id のフォーマットルールを確認（outlier_data と missing_data を使用）**
outlier_data には外れ値以外のデータ異常が含まれておらず、missing_data には欠損値以外のデータ異常が含まれていないため、これらのデータを利用してorder_idのフォーマットルールを抽出する。

In [21]:
# 異常値データ（outlier_data）と欠損データ（missing_data）から支店ごとの order_id ルールを取得
outlier_order_branch = outlier_data[['order_id', 'branch_code']]
missing_order_branch = missing_data[['order_id', 'branch_code']]

# 異常値データと欠損データを統合
order_branch_rules = pd.concat([outlier_order_branch, missing_order_branch])

print("支店ごとの order_id ルール:")
print(order_branch_rules)

支店ごとの order_id ルール:
      order_id branch_code
0    ORDX07128          BK
1    ORDY04820          TP
2    ORDI06977          NS
3    ORDC04090          NS
4    ORDC04240          NS
..         ...         ...
495  ORDK05906          BK
496  ORDI06900         NaN
497  ORDZ10456          NS
498  ORDB10191          TP
499  ORDJ06590          TP

[1000 rows x 2 columns]


In [22]:
# order_id のプレフィックス（ORD+英字部分）を抽出
def extract_order_prefix(order_id):
    character = re.search(r'(ORD[A-Z])', order_id)
    return character.group(1)

# order_id のプレフィックスのみ取得
order_branch_rules['order_id'] = order_branch_rules['order_id'].apply(
    extract_order_prefix)

# 支店ごとに order_id のパターンを集約
order_branch_set = order_branch_rules.groupby('branch_code').agg(set)

print("支店ごとの order_id のプレフィックスルール:")
print(order_branch_set)

支店ごとの order_id のプレフィックスルール:
                       order_id
branch_code                    
BK           {ORDX, ORDA, ORDK}
NS           {ORDC, ORDZ, ORDI}
TP           {ORDB, ORDJ, ORDY}


**フォーマットルールをもとに支店コードの間違いを修正（dierty_dataの修正）**

In [23]:
# 支店コードを修正する関数
def fix_branch_code(data_frame, idx_list = fixed_index):
    """
    order_id のパターンに基づいて、誤った支店コードを修正する。

    Args:
        data_frame (pd.DataFrame): 注文データの DataFrame
        idx_list (list): 修正された行のインデックスを記録するリスト
    """
    for idx, row in data_frame.iterrows():
        order_prefix = row['order_id']
        branch = row['branch_code']

        if re.search(r'ORD[XKA]', order_prefix) != None and branch in ['NS','TP']:
            data_frame.at[idx,'branch_code'] = 'BK'
            idx_list.append(idx)
        if re.search(r'ORD[ICZ]', order_prefix) != None and branch in ['BK','TP']:
            data_frame.at[idx,'branch_code'] = 'NS'
            idx_list.append(idx)
        if re.search(r'ORD[JYB]', order_prefix) != None and branch in ['BK','NS']:
            data_frame.at[idx,'branch_code'] = 'TP'
            idx_list.append(idx)

# 支店コードの修正を適用
fix_branch_code(dirty_data)

# 修正後の支店コードと order_id を確認
print("修正後の order_id と 支店コード:")
print(dirty_data[['order_id', 'branch_code']])

# 修正された行の総数を表示
print(f"\n合計 {len(set(fixed_index))} 件の配達履歴を修正しました。")

修正後の order_id と 支店コード:
      order_id branch_code
0    ORDI10384          NS
1    ORDB06250          TP
2    ORDC03391          NS
3    ORDZ10295          NS
4    ORDK02931          BK
..         ...         ...
495  ORDK02085          BK
496  ORDB09078          TP
497  ORDC10427          NS
498  ORDI10080          NS
499  ORDY02686          TP

[500 rows x 2 columns]

合計 102 件の配達履歴を修正しました。


## 2.4 メニュー情報の修正

### メニュー項目を抽出
outlier_data には外れ値以外のデータ異常が含まれていないため、このデータを利用してメニュー項目を抽出する。

In [24]:
# 'order_items 列の基本的な情報
print("【order_items列の基本統計情報】")
print(outlier_data['order_items'].describe())

【order_items列の基本統計情報】
count                                500
unique                               498
top       [('Steak', 7), ('Chicken', 1)]
freq                                   2
Name: order_items, dtype: object


In [25]:
# outlier_data のコピーを作成
outlier_data_copy = outlier_data.copy()

# 'order_items' 列を文字列からリストに変換
outlier_data_copy['order_items'] = outlier_data_copy['order_items'].apply(ast.literal_eval)
print(type(outlier_data_copy['order_items'].iloc[0]))  # 確認

<class 'list'>


In [26]:
# 朝食・昼食・夕食ごとのメニュー項目を取得する関数
def get_unique_items(df, meal_type):
    """
    指定した食事区分（朝食・昼食・夕食）のメニュー項目を取得する。

    Args:
        df (pd.DataFrame): 注文データを含む DataFrame。
        meal_type (str): 食事区分（"Breakfast", "Lunch", "Dinner" のいずれか）。

    Returns:
        list: 該当する食事区分のユニークなメニュー項目（昇順ソート）。
    """
    filtered_df = df.query(f'order_type == "{meal_type}"')
    item_lists = filtered_df['order_items'].to_list()
    unique_items = {item for sublist in item_lists for item, quantity in sublist}
    return sorted(unique_items)

# 各食事区分のメニュー項目を取得
breakfast_items = get_unique_items(outlier_data_copy, "Breakfast")
lunch_items = get_unique_items(outlier_data_copy, "Lunch")
dinner_items = get_unique_items(outlier_data_copy, "Dinner")

# 結果を表示
print(f'朝食のメニュー項目: {breakfast_items}')
print(f'昼食のメニュー項目: {lunch_items}')
print(f'夕食のメニュー項目: {dinner_items}')

朝食のメニュー項目: ['Cereal', 'Coffee', 'Eggs', 'Pancake']
昼食のメニュー項目: ['Burger', 'Chicken', 'Fries', 'Salad', 'Steak']
夕食のメニュー項目: ['Fish&Chips', 'Pasta', 'Salmon', 'Shrimp']


### メニューの単価を計算

**注文データの整理**

まず、注文内容（order_items）と注文価格（order_price）を食事区分ごとに取得する。

In [27]:
# 'order_items' 列を文字列からリストに変換
dirty_data['order_items'] = dirty_data['order_items'].apply(ast.literal_eval)

# 誤りが含まれる行のデータを取得（1 行 = 1 つのエラー）
order_details = []
for i in fixed_index:
    order_details.append([
        dirty_data['order_items'][i],
        dirty_data['order_type'][i],
        dirty_data['order_price'][i]
    ])

# 各注文リストをソート（アイテムの順序を統一）
for order in order_details:
    order[0].sort()

# 食事区分ごとに注文データを分類
breakfast_orders = [order for order in order_details if order[1] == 'Breakfast']
lunch_orders = [order for order in order_details if order[1] == 'Lunch']
dinner_orders = [order for order in order_details if order[1] == 'Dinner']

**アイテムごとに数値化後、線形代数を利用して単価を算出**

In [28]:
# 注文データを数値表現に変換する関数
def convert_items_to_numeric(order_list, item_catalog):
    item_dict = {item[0]: item[1] for item in order_list[0]}
    item_structured = {i: item_dict.get(i, 0) for i in item_catalog}
    return list(item_structured.values())

# 各食事区分のデータを数値に変換
breakfast_matrix = [(convert_items_to_numeric(order, breakfast_items), order[2]) for order in breakfast_orders]
lunch_matrix = [(convert_items_to_numeric(order, lunch_items), order[2]) for order in lunch_orders]
dinner_matrix = [(convert_items_to_numeric(order, dinner_items), order[2]) for order in dinner_orders]

# メニューの単価を算出する関数
def calculate_unit_prices(order_matrix, item_catalog):
    """
    メニューの単価を線形代数を用いて求める。

    行列が特異な場合は、最小二乗法を適用する。

    Args:
        order_matrix (list): 数値化された注文データと価格のリスト。
        item_catalog (list): メニューのアイテムリスト。

    Returns:
        np.array: メニュー項目ごとの単価。
    """
    item_list = [order[0] for order in order_matrix]
    price_list = [order[1] for order in order_matrix]

    item_matrix = np.array(item_list)
    price_vector = np.array(price_list)

    try:
        # 正則な行列であれば通常の解法を適用
        unit_prices = np.linalg.solve(item_matrix, price_vector)
    except np.linalg.LinAlgError:
        # 行列が特異な場合は最小二乗法を使用
        unit_prices, _, _, _ = np.linalg.lstsq(item_matrix, price_vector, rcond=None)

    return unit_prices

# 朝食・昼食・夕食の単価を計算
breakfast_prices = calculate_unit_prices(breakfast_matrix, breakfast_items)
lunch_prices = calculate_unit_prices(lunch_matrix, lunch_items)
dinner_prices = calculate_unit_prices(dinner_matrix, dinner_items)

# 計算結果を表示
print(f'朝食のメニュー: {breakfast_items}\n単価: {breakfast_prices}')
print(f'\n昼食のメニュー: {lunch_items}\n単価: {lunch_prices}')
print(f'\n夕食のメニュー: {dinner_items}\n単価: {dinner_prices}')

朝食のメニュー: ['Cereal', 'Coffee', 'Eggs', 'Pancake']
単価: [21.    7.5  22.   24.25]

昼食のメニュー: ['Burger', 'Chicken', 'Fries', 'Salad', 'Steak']
単価: [31.  32.  12.  17.2 45. ]

夕食のメニュー: ['Fish&Chips', 'Pasta', 'Salmon', 'Shrimp']
単価: [35.  27.5 41.  54. ]


### 誤った注文商品の修正

**メニューの単価辞書を作成**

In [29]:
# 各食事区分の単価を辞書形式にまとめる
breakfast_price_dict = dict(zip(breakfast_items, breakfast_prices))
lunch_price_dict = dict(zip(lunch_items, lunch_prices))
dinner_price_dict = dict(zip(dinner_items, dinner_prices))

# すべての単価情報を統合
combined_price_dict = {}
combined_price_dict.update(breakfast_price_dict)
combined_price_dict.update(lunch_price_dict)
combined_price_dict.update(dinner_price_dict)

# 単価を小数点 2 桁で丸める
for item in combined_price_dict:
    combined_price_dict[item] = np.around(combined_price_dict[item], decimals=2)

# 単価辞書を表示
print("メニュー単価の辞書:")
combined_price_dict

メニュー単価の辞書:


{'Cereal': 21.0,
 'Coffee': 7.5,
 'Eggs': 22.0,
 'Pancake': 24.25,
 'Burger': 31.0,
 'Chicken': 32.0,
 'Fries': 12.0,
 'Salad': 17.2,
 'Steak': 45.0,
 'Fish&Chips': 35.0,
 'Pasta': 27.5,
 'Salmon': 41.0,
 'Shrimp': 54.0}

**誤ったメニュー項目の検出**

order_items 列のデータをチェックし、食事区分ごとに誤ったメニュー項目が含まれているかどうかを確認する。

In [30]:
# 注文内容が正しいか判定する関数
def check_invalid_items(order_list, valid_item_dict):
    """
    注文の各メニューが正しいかどうかを判定する。

    Args:
        order_list (list of tuples): 注文内容（(メニュー, 数量) のリスト）。
        valid_item_dict (dict): 許可されたメニューの辞書。

    Returns:
        list: 各メニューの正誤（True: 正しい, False: 誤り）。
    """
    return [item[0] in valid_item_dict for item in order_list]

# 各注文の誤ったメニューを特定する関数
def identify_invalid_orders(data_frame):
    """
    各注文に誤ったメニューが含まれているかを判定し、新しい列にフラグを追加する。

    Args:
        data_frame (pd.DataFrame): 注文データを含む DataFrame。
    """
    data_frame['invalid_item_flags'] = None  # Initialize the column

    for idx, row in data_frame.iterrows():
        if row['order_type'] == "Breakfast":
            data_frame.at[idx, 'invalid_item_flags'] = check_invalid_items(row['order_items'], breakfast_price_dict)
        elif row['order_type'] == "Lunch":
            data_frame.at[idx, 'invalid_item_flags'] = check_invalid_items(row['order_items'], lunch_price_dict)
        elif row['order_type'] == "Dinner":
            data_frame.at[idx, 'invalid_item_flags'] = check_invalid_items(row['order_items'], dinner_price_dict)

# 誤ったメニュー項目を特定する関数を適用
identify_invalid_orders(dirty_data)

# 誤ったメニューが１つ以上含まれている注文を抽出
invalid_orders = dirty_data[dirty_data['invalid_item_flags'].apply(lambda x: False in x)]

# 結果を表示
print("誤ったメニュー項目が含まれている注文:")
invalid_orders

誤ったメニュー項目が含まれている注文:


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,invalid_item_flags
10,ORDI01942,2018-03-06,10:01:41,Breakfast,NS,"[(Cereal, 3), (Fries, 5), (Coffee, 3)]",206.75,-37.816075,144.983844,0,8.661,13.361177,"[True, False, True]"
47,ORDC02271,2018-05-01,11:53:14,Breakfast,NS,"[(Cereal, 3), (Chicken, 6), (Coffee, 3)]",217.50,-37.819224,144.946592,0,9.644,14.801910,"[True, False, True]"
64,ORDA08025,2018-07-19,14:35:29,Lunch,BK,"[(Fries, 7), (Steak, 9), (Chicken, 3), (Cereal...",647.00,-37.805698,144.977241,0,6.442,11.777423,"[True, True, True, False]"
67,ORDY08329,2018-08-20,15:36:20,Lunch,TP,"[(Chicken, 10), (Steak, 6), (Eggs, 1), (Fries,...",767.20,-37.801431,144.979769,0,10.246,13.742179,"[True, True, False, True, True]"
87,ORDA01657,2018-07-14,14:25:21,Lunch,BK,"[(Salmon, 9), (Chicken, 2), (Burger, 2), (Frie...",625.40,-37.817756,145.007801,0,4.058,12.212661,"[False, True, True, True, True]"
101,ORDX02026,2018-03-23,10:42:15,Breakfast,BK,"[(Burger, 2), (Coffee, 5), (Pancake, 1)]",103.75,-37.819978,144.985319,0,6.277,11.012713,"[False, True, True]"
109,ORDJ04430,2018-07-12,14:25:21,Lunch,TP,"[(Burger, 3), (Pancake, 2), (Salad, 7)]",237.40,-37.818781,145.000767,0,10.659,13.997161,"[True, False, True]"
112,ORDK09046,2018-06-15,13:24:30,Lunch,BK,"[(Salad, 7), (Steak, 5), (Fries, 3), (Cereal, ...",917.40,-37.813490,144.965667,0,7.490,12.837434,"[True, True, True, False, True]"
114,ORDA10554,2018-12-05,19:09:17,Dinner,BK,"[(Fish&Chips, 8), (Pancake, 1), (Pasta, 6)]",499.00,-37.822615,144.958100,0,8.591,15.924244,"[True, False, True]"
124,ORDJ08214,2018-10-24,17:48:10,Dinner,TP,"[(Steak, 3), (Salmon, 3)]",205.50,-37.820403,144.966611,0,7.468,11.916155,"[False, True]"


**誤った注文商品の修正**

上記の関数で特定した誤った注文内容を、以下の手順で修正する：
- 間違ったメニューを特定して削除
- 正しいメニュー名に置き換える
- 単価情報を利用して合計金額を調整

In [31]:
# 誤ったメニューを修正する関数
def correct_order_items(row, valid_item_dict, price_dict=combined_price_dict):
    """
    誤ったメニューの記述を修正し、正しいメニュー名に置き換える。

    Args:
        row (pd.Series): 修正対象の注文データ（DataFrame の1行）。
        valid_item_dict (dict): 正しいメニューの辞書。
        price_dict (dict): メニュー名と単価の対応辞書。

    Returns:
        list: 修正済みの注文内容。
    """
    # 注文情報を取得
    order_items = row['order_items']
    total_price = row['order_price']

    # 誤ったメニュー項目を特定
    invalid_items = [item for item in order_items if item[0] not in valid_item_dict]

    # 誤ったメニューを削除
    corrected_items = [item for item in order_items if item not in invalid_items]

    # メニューの単価を取得し、数値データに変換
    converted_items = [(price_dict.get(item[0]), item[1]) for item in corrected_items]

    # 修正後の合計金額を計算
    numeric_totals = [item_price * quantity for item_price, quantity in converted_items]
    price_difference = abs(sum(numeric_totals) - total_price)  # 差額を計算

    # 差額を補填するための単価を計算
    if sum([item[1] for item in invalid_items]) > 0:
        ratio_per_unit = price_difference / sum([item[1] for item in invalid_items])
        ratio_per_unit = np.around(ratio_per_unit, decimals=2)

        # もし ratio_per_unit が price_dict の values にない場合は最も近い値を探す
        if ratio_per_unit not in price_dict.values():
            closest_item = min(price_dict.keys(), key=lambda k: abs(price_dict[k] - ratio_per_unit))
        else:
            closest_item = list(price_dict.keys())[list(price_dict.values()).index(ratio_per_unit)]

        # 修正済みの注文リストに追加
        corrected_items.append((closest_item, invalid_items[0][1]))

    return corrected_items

In [32]:
# 誤った注文を修正し、修正したレコードのインデックスを記録する関数
def fix_invalid_orders(data_frame, fixed_index_list):
    """
    誤ったメニュー項目を修正し、変更したレコードのインデックスを記録する。

    Args:
        data_frame (pd.DataFrame): 修正対象のデータセット。
        fixed_index_list (list): 修正されたレコードのインデックスを保存するリスト。
    """
    for idx, row in data_frame.iterrows():
        if False in row['invalid_item_flags']:  # 誤りが含まれている場合
            fixed_index_list.append(idx)

            if row['order_type'] == 'Breakfast':
                data_frame.at[idx, 'order_items'] = correct_order_items(row, breakfast_price_dict)
            elif row['order_type'] == 'Lunch':
                data_frame.at[idx, 'order_items'] = correct_order_items(row, lunch_price_dict)
            else:
                data_frame.at[idx, 'order_items'] = correct_order_items(row, dinner_price_dict)

# 修正を適用
fixed_index_items = []
fix_invalid_orders(dirty_data, fixed_index_items)

# 修正後のデータを確認
corrected_orders = dirty_data.loc[dirty_data.index.isin(set(fixed_index_items))]
print("修正済みの注文:")
print(corrected_orders.head(10))

# 修正インデックスリストを更新
fixed_index.extend(fixed_index_items)

# インデックスに重複がないか確認
print("修正後のインデックスがユニークか:", len(fixed_index) == len(set(fixed_index)))

修正済みの注文:
      order_id        date      time order_type branch_code  \
10   ORDI01942  2018-03-06  10:01:41  Breakfast          NS   
47   ORDC02271  2018-05-01  11:53:14  Breakfast          NS   
64   ORDA08025  2018-07-19  14:35:29      Lunch          BK   
67   ORDY08329  2018-08-20  15:36:20      Lunch          TP   
87   ORDA01657  2018-07-14  14:25:21      Lunch          BK   
101  ORDX02026  2018-03-23  10:42:15  Breakfast          BK   
109  ORDJ04430  2018-07-12  14:25:21      Lunch          TP   
112  ORDK09046  2018-06-15  13:24:30      Lunch          BK   
114  ORDA10554  2018-12-05  19:09:17     Dinner          BK   
124  ORDJ08214  2018-10-24  17:48:10     Dinner          TP   

                                           order_items  order_price  \
10            [(Cereal, 3), (Coffee, 3), (Pancake, 5)]       206.75   
47               [(Cereal, 3), (Coffee, 3), (Eggs, 6)]       217.50   
64   [(Fries, 7), (Steak, 9), (Chicken, 3), (Burger...       647.00   
67   [(Chicke

### 注文価格 (order_price) の誤りを修正

In [33]:
# 注文価格 (`order_price`) を修正する関数
def fix_order_price(data_frame, fixed_index_list, price_dict=combined_price_dict):
    """
    修正済みの注文内容 (`order_items`) をもとに、誤った `order_price` を修正する。

    Args:
        data_frame (pd.DataFrame): 注文データを含む DataFrame。
        fixed_index_list (list): 修正されたレコードのインデックスを保存するリスト。
        price_dict (dict): メニュー名と単価の対応辞書。
    """
    for idx, row in data_frame.iterrows():
        corrected_items = row['order_items']  # 前のステップで修正された order_items を取得

        # 各メニューの単価を取得し、合計金額を計算（単価 × 数量）
        calculated_price = sum(price_dict.get(item[0], 0) * item[1] for item in corrected_items)

        # order_price が誤っている場合、修正する
        if np.around(calculated_price, decimals=2) != row['order_price']:
            data_frame.at[idx, 'order_price'] = np.around(calculated_price, decimals=2)
            fixed_index_list.append(idx)

# 修正を適用
fixed_price_indexes = []
fix_order_price(dirty_data, fixed_price_indexes)

# 修正された注文価格のデータを取得し、修正結果を確認
corrected_prices = dirty_data.loc[dirty_data.index.isin(set(fixed_price_indexes))]
print("修正された注文価格:")
corrected_prices[['order_items', 'order_price']].head(10)

修正された注文価格:


,order_items,order_price
20,"[(Eggs, 8), (Cereal, 9), (Pancake, 5), (Coffee...",523.75
26,"[(Coffee, 5), (Pancake, 7), (Cereal, 9), (Eggs...",528.25
52,"[(Fries, 5), (Burger, 1), (Salad, 7), (Steak, ...",666.40
56,"[(Salad, 4), (Chicken, 3), (Burger, 1), (Steak...",333.80
57,"[(Coffee, 6), (Eggs, 9), (Pancake, 6), (Cereal...",493.50
72,"[(Fish&Chips, 2), (Shrimp, 4), (Salmon, 8), (P...",696.50
85,"[(Chicken, 5), (Salad, 9), (Burger, 3)]",407.80
96,"[(Pancake, 6), (Coffee, 4), (Cereal, 9)]",364.50
122,"[(Fries, 3), (Salad, 9)]",190.80
166,"[(Fish&Chips, 2), (Shrimp, 8), (Salmon, 7)]",789.00


In [34]:
# 修正インデックスを更新
fixed_index.extend(fixed_price_indexes)

# 修正インデックスがユニークか確認（True なら重複なし）
print("修正後のインデックスがユニークか:", len(fixed_index) == len(set(fixed_index)))

# 不要な列（ invalid_item_flags ）を削除
dirty_data = dirty_data.drop(columns=['invalid_item_flags'])

# 修正が適用された注文の合計件数を表示
print("合計修正件数:", len(fixed_index))

修正後のインデックスがユニークか: True
合計修正件数: 176


## 2.5 位置情報の修正

本来、**オーストラリアの緯度 (latitude) は -37 〜 -38、経度 (longitude) は約 144 であるべき**だが、データを確認すると緯度の最大値が 144 、経度の最小値が -37 になっている。

加えて、異常な緯度を持つデータの緯度と、異常な経度を持つデータの経度を確認すると、
それぞれの値がほぼ 144 と -37 で一致している。このことから、**一部のデータで緯度と経度が入れ替わっている可能性が高い**。

さらに、異常な緯度データを調べると、**緯度が約 37 になっているデータも存在しており、これはマイナスを付け忘れた可能性が高い**。

修正の流れ
- 緯度と経度が入れ替わっているデータを修正（正しい列に配置）
- すべての正の緯度を負の値に統一（-37 〜 -38 の範囲に修正）

In [35]:
# 'customer_lat', 'customer_lon' 列の基本的な情報
dirty_data[['customer_lat', 'customer_lon']].describe()

,customer_lat,customer_lon
count,500.000000,500.000000
mean,-30.754372,143.505455
std,25.337195,16.299611
min,-37.840654,-37.823004
25%,-37.818719,144.952627
50%,-37.812863,144.965031
75%,-37.805265,144.982501
max,144.986713,145.019877


In [36]:
# 経度の値が異常なデータを特定（オーストラリアの経度は通常 約144）
invalid_lon_rows = dirty_data.query('customer_lon < 140')

# 修正前の異常なデータを表示
print("修正前：緯度の値が異常な注文")
print(invalid_lon_rows[['customer_lat', 'customer_lon']])
print(f"\n修正対象のインデックス: {invalid_lon_rows.index.tolist()}")

修正前：緯度の値が異常な注文
     customer_lat  customer_lon
136    144.986713    -37.810674
173    144.979357    -37.823004
358    144.984815    -37.822654
427    144.936078    -37.814016

修正対象のインデックス: [136, 173, 358, 427]


In [37]:
# 緯度の値が異常なデータを特定（オーストラリアの緯度は通常 -37 〜 -38）
invalid_lat_rows = dirty_data.query('customer_lat > - 35')

# 修正前の異常なデータを表示
print("修正前：緯度の値が異常な注文")
print(invalid_lat_rows[['customer_lat', 'customer_lon']])
print(f"\n修正対象のインデックス: {invalid_lat_rows.index.tolist()}")

修正前：緯度の値が異常な注文
     customer_lat  customer_lon
0       37.806118    144.934214
9       37.808648    144.952979
30      37.800515    144.959175
48      37.811725    144.956322
49      37.812838    144.973561
50      37.810784    145.016324
65      37.821443    144.945078
88      37.810910    145.004160
89      37.816011    145.005755
103     37.802746    144.994296
136    144.986713    -37.810674
142     37.817898    144.948546
144     37.819050    144.976880
158     37.821143    144.954898
173    144.979357    -37.823004
198     37.824398    144.944631
200     37.811331    144.969293
204     37.816200    145.001961
229     37.810004    144.995305
232     37.815192    144.959608
235     37.810034    144.935960
239     37.799995    144.974023
249     37.813259    144.962635
285     37.804604    144.910296
301     37.802477    144.963315
313     37.810718    145.002339
318     37.809540    144.972050
343     37.813351    144.965110
354     37.821793    144.990754
358    144.984815    -37.

### 緯度・経度の値が入れ替わっている問題

In [38]:
# 緯度と経度を入れ替える処理
dirty_data.loc[invalid_lon_rows.index, ['customer_lat', 'customer_lon']] = \
    dirty_data.loc[invalid_lon_rows.index, ['customer_lon', 'customer_lat']].values

# 修正されたデータを取得
lat_lon_switched = dirty_data.loc[invalid_lon_rows.index, ['customer_lat', 'customer_lon']]

# 修正結果を表示
print("修正後の緯度・経度データ:")
print(lat_lon_switched)


# 修正された行のインデックスを記録
#fixed_index.extend(invalid_lon_rows.index.tolist())

# 修正が適用された行の合計を表示
#print(f"\n配達履歴を修正したトータル注文数: {len(fixed_index)} 件")

修正後の緯度・経度データ:
     customer_lat  customer_lon
136    -37.810674    144.986713
173    -37.823004    144.979357
358    -37.822654    144.984815
427    -37.814016    144.936078


### 緯度の値にマイナスをつけ忘れている問題

In [39]:
# 緯度の値に不整合があるデータを特定（緯度が -1 より大きいもの）
incorrect_latitude_rows = dirty_data.query('customer_lat > -1')

# 不正な緯度の符号を修正（正の値を負に変換）
dirty_data.loc[dirty_data['customer_lat'] > 1, 'customer_lat'] *= -1

# 修正後のデータを取得
corrected_latitude_rows = dirty_data.loc[incorrect_latitude_rows.index, ['customer_lat', 'customer_lon']]

# 修正後の緯度データを表示
print("修正後の緯度データ:")
print(corrected_latitude_rows)

修正後の緯度データ:
     customer_lat  customer_lon
0      -37.806118    144.934214
9      -37.808648    144.952979
30     -37.800515    144.959175
48     -37.811725    144.956322
49     -37.812838    144.973561
50     -37.810784    145.016324
65     -37.821443    144.945078
88     -37.810910    145.004160
89     -37.816011    145.005755
103    -37.802746    144.994296
142    -37.817898    144.948546
144    -37.819050    144.976880
158    -37.821143    144.954898
198    -37.824398    144.944631
200    -37.811331    144.969293
204    -37.816200    145.001961
229    -37.810004    144.995305
232    -37.815192    144.959608
235    -37.810034    144.935960
239    -37.799995    144.974023
249    -37.813259    144.962635
285    -37.804604    144.910296
301    -37.802477    144.963315
313    -37.810718    145.002339
318    -37.809540    144.972050
343    -37.813351    144.965110
354    -37.821793    144.990754
359    -37.819607    144.952028
373    -37.819565    144.969116
393    -37.820849    144.9551

In [40]:
# 修正インデックスを更新
fixed_index.extend(list(incorrect_latitude_rows.index))

# 修正インデックスがユニークか確認（True なら重複なし）
print("修正後のインデックスがユニークか:", len(fixed_index) == len(set(fixed_index)))

# 修正が適用された注文の合計件数を表示
print("合計修正件数:", len(fixed_index))

修正後のインデックスがユニークか: True
合計修正件数: 213


## 2.6 配達距離の修正

**前提**
- Dijkstra（ダイクストラ）アルゴリズムを用いて、顧客の位置からレストラン（支店）までの最短距離を計算している

**支店データ・ノードデータとの統合**

正しい配達距離を算出するため、以下のデータを統合

- dirty_data（元の注文データ）
- branch_data（支店の緯度・経度情報）
- node_data（各ノードの緯度・経度・ノードID情報）

まず、注文データと支店データを結合し、支店の緯度・経度情報を取得する。
次に、node_data を利用して、顧客の位置（customer_node）と支店の位置（branch_node）を取得する。

In [41]:
# dirty_data と branch_data を結合して支店の緯度・経度を取得
dirty_data_merged = pd.merge(
    dirty_data,
    branch_data[['branch_code', 'branch_lat', 'branch_lon']],
    on='branch_code',
    how='left'
)

# dirty_data を node_data と結合して顧客のノードを取得
dirty_data_merged = pd.merge(
    dirty_data_merged,
    node_data[['lat', 'lon', 'node']],
    left_on=['customer_lat', 'customer_lon'],
    right_on=['lat', 'lon'],
    how='left'
).rename(columns={'node': 'customer_node'})  # ノード名を customer_node に変更

# branch_data を node_data と結合して支店のノードを取得
branch_nodes = pd.merge(
    branch_data,
    node_data[['lat', 'lon', 'node']],
    left_on=['branch_lat', 'branch_lon'],
    right_on=['lat', 'lon'],
    how='left'
).rename(columns={'node': 'branch_node'})  # ノード名を branch_node に変更

# 支店ノード情報を dirty_data_merged に統合
dirty_data_merged = pd.merge(
    dirty_data_merged,
    branch_nodes[['branch_code', 'branch_node']],
    on='branch_code',
    how='left'
)

dirty_data_merged.head(5)

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,branch_lat,branch_lon,lat,lon,customer_node,branch_node
0,ORDI10384,2018-01-16,08:30:25,Breakfast,NS,"[(Coffee, 6), (Cereal, 6)]",171.0,-37.806118,144.934214,1,10.009,7.788453,-37.773803,144.983647,-37.806118,144.934214,579999252,2455254505
1,ORDB06250,2018-04-16,11:22:49,Breakfast,TP,"[(Eggs, 1), (Coffee, 6), (Pancake, 2)]",115.5,-37.810433,144.968155,1,8.762,11.704791,-37.861835,144.905716,-37.810433,144.968155,6167236666,1390575046
2,ORDC03391,2018-11-12,18:28:43,Dinner,NS,"[(Pasta, 8), (Shrimp, 5), (Salmon, 8), (Fish&C...",1028.0,-37.803893,144.978943,0,7.129,13.074209,-37.773803,144.983647,-37.803893,144.978943,2457712029,2455254505
3,ORDZ10295,2018-08-25,15:46:28,Lunch,NS,"[(Burger, 4), (Steak, 6)]",394.0,-37.820483,144.981167,1,9.603,16.994845,-37.773803,144.983647,-37.820483,144.981167,577457184,2455254505
4,ORDK02931,2018-12-14,19:29:34,Dinner,BK,"[(Salmon, 10), (Pasta, 1)]",437.5,-37.817963,144.932101,0,10.610,17.507001,-37.815834,145.046450,-37.817963,144.932101,6197209819,1889485053


**Dijkstra アルゴリズムを使用して最短距離を計算**

顧客の位置（customer_node）と支店の位置（branch_node）の最短距離をDijkstra アルゴリズムを用いて算出する

In [42]:
# edge_data を使用してグラフを作成（距離を重みとして設定）
G = nx.Graph()
for idx, row in edge_data.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['distance(m)'])

# 最短距離を計算する関数
def calculate_distance(row):
    """
    顧客のノードと支店のノードの最短距離を求める（Dijkstra アルゴリズム使用）。

    Args:
        row (pd.Series): DataFrame の1行（customer_node, branch_node を含む）

    Returns:
        float: 最短距離（km単位）
    """
    path_length = nx.dijkstra_path_length(G, source=row['customer_node'], target=row['branch_node'], weight='weight')
    return path_length / 1000  # m → km に変換

# 計算結果を新しい列に保存
dirty_data_merged['calculated_distance_KM'] = dirty_data_merged.apply(calculate_distance, axis=1)

# 結果を表示
dirty_data_merged['calculated_distance_KM']

0      10.009
1       8.762
2       7.129
3       9.603
4      10.610
        ...  
495     8.632
496     7.846
497     9.293
498     9.035
499     7.853
Name: calculated_distance_KM, Length: 500, dtype: float64

**配達距離の誤りを修正**

データに記録されている配達距離 distance_to_customer_KM と、Dijkstra アルゴリズムで計算した calculated_distance_KM を比較し、異なる場合は修正

In [43]:
# 配達距離が異なるデータを修正
for index, row in dirty_data_merged.iterrows():
    original_distance = row['distance_to_customer_KM']
    calculated_distance = row['calculated_distance_KM']

    # 記録されている距離と計算距離が異なる場合、修正
    if original_distance != calculated_distance:
        fixed_index.append(index)
        dirty_data_merged.at[index, 'distance_to_customer_KM'] = calculated_distance
        print(f"注文 {index}: "
              f"元の値: {original_distance:.2f} km → 修正後: {calculated_distance:.2f} km")

# 修正した件数を出力
print(f"\n修正された注文の合計数: {len(fixed_index)} 件")

注文 7: 元の値: 10.03 km → 修正後: 11.76 km
注文 13: 元の値: 12.05 km → 修正後: 8.59 km
注文 25: 元の値: 8.05 km → 修正後: 8.80 km
注文 27: 元の値: 10.61 km → 修正後: 4.24 km
注文 34: 元の値: 11.76 km → 修正後: 8.84 km
注文 41: 元の値: 9.25 km → 修正後: 9.22 km
注文 42: 元の値: 10.25 km → 修正後: 8.01 km
注文 59: 元の値: 7.13 km → 修正後: 8.40 km
注文 60: 元の値: 10.18 km → 修正後: 9.47 km
注文 66: 元の値: 7.62 km → 修正後: 8.58 km
注文 84: 元の値: 8.51 km → 修正後: 9.60 km
注文 91: 元の値: 8.67 km → 修正後: 5.66 km
注文 102: 元の値: 8.36 km → 修正後: 9.80 km
注文 106: 元の値: 8.83 km → 修正後: 8.13 km
注文 129: 元の値: 9.16 km → 修正後: 9.43 km
注文 150: 元の値: 8.16 km → 修正後: 7.08 km
注文 151: 元の値: 8.01 km → 修正後: 8.55 km
注文 155: 元の値: 10.51 km → 修正後: 7.67 km
注文 167: 元の値: 10.12 km → 修正後: 11.20 km
注文 170: 元の値: 10.61 km → 修正後: 8.79 km
注文 190: 元の値: 9.06 km → 修正後: 7.08 km
注文 209: 元の値: 8.35 km → 修正後: 9.65 km
注文 212: 元の値: 9.82 km → 修正後: 10.41 km
注文 247: 元の値: 8.61 km → 修正後: 9.70 km
注文 248: 元の値: 8.07 km → 修正後: 7.38 km
注文 281: 元の値: 7.45 km → 修正後: 9.77 km
注文 283: 元の値: 9.70 km → 修正後: 11.32 km
注文 315: 元の値: 7.45 km → 修正後: 

In [44]:
# 不要な列を削除
dirty_data_filtered = dirty_data_merged.drop(columns = ['branch_lat', 'branch_lon',
       'lat', 'lon', 'customer_node', 'branch_node', 'calculated_distance_KM'])
dirty_data_filtered.columns

Index(['order_id', 'date', 'time', 'order_type', 'branch_code', 'order_items',
       'order_price', 'customer_lat', 'customer_lon', 'customerHasloyalty?',
       'distance_to_customer_KM', 'delivery_fee'],
      dtype='object')

## 2.7 会員ステータスの修正

**前提条件**
- delivery_fee（配達料金）の値は常に正しい
- customerHasloyalty?（ロイヤルティ会員ステータス）が 1（会員）の場合、配達料金は 50% 割引される。0（非会員）の場合は、delivery_fee がそのまま適用される
- 配達料金は、支店ごとに異なる計算方法で決定され、以下の要素に基づいて線形的に変化する：
    - 平日・週末（0: 平日、1: 週末）
    - 時間帯（0: 朝、1: 昼、2: 夜）
    - 配達距離（レストランから顧客までの距離）

**前処理**

まず、日付と注文区分（order_type）を変換し、元の配達料金 (original_price) を求める。

In [45]:
# データをコピー
cleaned_data = dirty_data_filtered.copy()
train_data = missing_data.copy()

In [46]:
# 平日（0）・週末（1）を判定する関数
def categorize_day(date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d')
    if date.weekday() < 5:
        return 0
    else:
        return 1

# 注文区分を数値に変換する関数
def order_type_to_numeric(order_type):
    order_type = order_type.capitalize()
    if order_type == 'Breakfast':
        return 0
    elif order_type == 'Lunch':
        return 1
    elif order_type == 'Dinner':
        return 2

# ロイヤルティ会員ステータスを考慮した元の配達料金を計算する関数
def calculate_original_price(row):
    if row['customerHasloyalty?'] == 1:
        return row['delivery_fee'] * 2
    else:
        return row['delivery_fee']

# 欠損値のある行を削除
train_data = train_data.dropna()

# 変換処理の適用
train_data['date'] = train_data['date'].apply(categorize_day)
train_data['order_type_numeric'] = train_data['order_type'].apply(order_type_to_numeric)
train_data['original_price'] = train_data.apply(calculate_original_price, axis=1)

cleaned_data['date'] = cleaned_data['date'].apply(categorize_day)
cleaned_data['order_type_numeric'] = cleaned_data['order_type'].apply(order_type_to_numeric)
cleaned_data['original_price'] = cleaned_data.apply(calculate_original_price, axis=1)

**支店ごとの回帰モデルを構築**

各支店 (branch_code) ごとに線形回帰モデルを作成し、予測精度を評価する。

支店「BK」のモデル：

In [47]:
# "BK" 支店のデータを分割
train_bk = train_data.query('branch_code == "BK"').copy()
test_data = cleaned_data.loc[cleaned_data.index.isin(set(fixed_index))].copy()
test_bk = test_data.query('branch_code == "BK"').copy()

# 特徴量と目的変数の設定
feature_columns = ['date', 'order_type_numeric', 'distance_to_customer_KM']
X_train_bk = train_bk[feature_columns]
y_train_bk = train_bk['original_price']

# 線形回帰モデルを学習
model_bk = LinearRegression()
model_bk.fit(X_train_bk, y_train_bk)

# モデルの評価
train_bk['predicted_price'] = model_bk.predict(X_train_bk)
train_r2 = model_bk.score(X_train_bk, y_train_bk)
train_mse = mean_squared_error(y_train_bk, train_bk['predicted_price'])

print("<BK支店 - 訓練データ>")
print(f"R-squared: {train_r2:.4f}")
print(f"MSE: {train_mse:.4f}")

# テストデータの評価
test_bk['predicted_price'] = model_bk.predict(test_bk[feature_columns])
test_r2 = r2_score(test_bk['original_price'], test_bk['predicted_price'])
test_mse = mean_squared_error(test_bk['original_price'], test_bk['predicted_price'])

print("\n<BK支店 - テストデータ>")
print(f"R-squared: {test_r2:.4f}")
print(f"MSE: {test_mse:.4f}")

<BK支店 - 訓練データ>
R-squared: 0.9738
MSE: 0.1274

<BK支店 - テストデータ>
R-squared: 0.9756
MSE: 0.1033


支店「NS」「TP」のモデル：

支店「NS」「TP」も同様の手順でモデルを作成

In [48]:
# "NS"支店のモデル
train_ns = train_data.query('branch_code == "NS"').copy()
test_ns = test_data.query('branch_code == "NS"').copy()

X_train_ns = train_ns[feature_columns]
y_train_ns = train_ns['original_price']

model_ns = LinearRegression()
model_ns.fit(X_train_ns, y_train_ns)

train_ns['predicted_price'] = model_ns.predict(X_train_ns)
test_ns['predicted_price'] = model_ns.predict(test_ns[feature_columns])

print("<NS支店 - 訓練データ>")
print(f"R-squared: {model_ns.score(X_train_ns, y_train_ns):.4f}")
print(f"MSE: {mean_squared_error(y_train_ns, train_ns['predicted_price']):.4f}")

print("\n<NS支店 - テストデータ>")
print(f"R-squared: {r2_score(test_ns['original_price'], test_ns['predicted_price']):.4f}")
print(f"MSE: {mean_squared_error(test_ns['original_price'], test_ns['predicted_price']):.4f}")

# "TP"支店のモデル
train_tp = train_data.query('branch_code == "TP"').copy()
test_tp = test_data.query('branch_code == "TP"').copy()

X_train_tp = train_tp[feature_columns]
y_train_tp = train_tp['original_price']

model_tp = LinearRegression()
model_tp.fit(X_train_tp, y_train_tp)

train_tp['predicted_price'] = model_tp.predict(X_train_tp)
test_tp['predicted_price'] = model_tp.predict(test_tp[feature_columns])

print("\n<TP支店 - 訓練データ>")
print(f"R-squared: {model_tp.score(X_train_tp, y_train_tp):.4f}")
print(f"MSE: {mean_squared_error(y_train_tp, train_tp['predicted_price']):.4f}")

print("\n<TP支店 - テストデータ>")
print(f"R-squared: {r2_score(test_tp['original_price'], test_tp['predicted_price']):.4f}")
print(f"MSE: {mean_squared_error(test_tp['original_price'], test_tp['predicted_price']):.4f}")

<NS支店 - 訓練データ>
R-squared: 0.9618
MSE: 0.0836

<NS支店 - テストデータ>
R-squared: 0.9457
MSE: 0.0890

<TP支店 - 訓練データ>
R-squared: 0.9467
MSE: 0.1049

<TP支店 - テストデータ>
R-squared: 0.9710
MSE: 0.0696


**ロイヤルティ会員ステータスの修正**

学習したモデルを用いて、誤った customerHasloyalty? を修正

In [49]:
# 各支店ごとのモデルを辞書にマッピング
branch_models = {
    "BK": model_bk,
    "NS": model_ns,
    "TP": model_tp
}

# ロイヤルティ会員ステータスを予測する関数
def predict_loyalty(row):
    branch = row['branch_code']

    # 該当する支店のモデルが存在するか確認
    if branch in branch_models:
        model = branch_models[branch]

        # 予測される配達料金を算出
        prediction = model.predict(pd.DataFrame([row[feature_columns]], columns=feature_columns))[0]
        row['predicted_delivery_fee'] = prediction

        # 予測値と実際の配達料金の差をもとに、会員判定の閾値を計算
        difference = prediction - row['delivery_fee']
        threshold = (prediction + difference) / 2

        # ロイヤルティ会員かどうかを判定
        row['customerHasloyalty?'] = 1 if row['delivery_fee'] <= threshold else 0

    return row

# データのコピーを作成し、ロイヤルティ会員ステータスを更新
fixed_data = cleaned_data.copy()
fixed_data = fixed_data.apply(predict_loyalty, axis=1)

In [50]:
# 不要な列を削除
output_data = fixed_data.drop(
    columns = ['order_type_numeric', 'original_price', 'predicted_delivery_fee'])

# 3 最終処理

## CSVファイルの出力

In [51]:
output_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDI10384,0,08:30:25,Breakfast,NS,"[(Coffee, 6), (Cereal, 6)]",171.00,-37.806118,144.934214,1,10.009,7.788453
1,ORDB06250,0,11:22:49,Breakfast,TP,"[(Eggs, 1), (Coffee, 6), (Pancake, 2)]",115.50,-37.810433,144.968155,0,8.762,11.704791
2,ORDC03391,0,18:28:43,Dinner,NS,"[(Pasta, 8), (Shrimp, 5), (Salmon, 8), (Fish&C...",1028.00,-37.803893,144.978943,0,7.129,13.074209
3,ORDZ10295,1,15:46:28,Lunch,NS,"[(Burger, 4), (Steak, 6)]",394.00,-37.820483,144.981167,0,9.603,16.994845
4,ORDK02931,0,19:29:34,Dinner,BK,"[(Salmon, 10), (Pasta, 1)]",437.50,-37.817963,144.932101,0,10.610,17.507001
...,...,...,...,...,...,...,...,...,...,...,...,...
495,ORDK02085,0,16:06:45,Dinner,BK,"[(Fish&Chips, 8), (Pasta, 9), (Salmon, 8)]",855.50,-37.800097,144.953861,0,8.632,15.506404
496,ORDB09078,0,17:27:53,Dinner,TP,"[(Fish&Chips, 3), (Pasta, 1), (Salmon, 6), (Sh...",810.50,-37.818915,144.964563,0,7.846,12.361560
497,ORDC10427,1,08:50:42,Breakfast,NS,"[(Pancake, 8), (Eggs, 1), (Coffee, 7)]",268.50,-37.821267,144.957756,0,9.293,16.673407
498,ORDI10080,0,10:11:49,Breakfast,NS,"[(Eggs, 2), (Pancake, 9), (Cereal, 3), (Coffee...",370.25,-37.818757,144.983192,0,9.035,13.514620


In [52]:
output_data.to_csv('dirty_data_solution.csv', index=False)

# 4 参考文献

My Data Talk (2022). Clean a Messy Date Column with Mixed Formats in Pandas. https://towardsdatascience.com/clean-a-messy-date-column-with-mixed-formats-in-pandas-1a88808edbf7
